In [1]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys
import string
import math
import heapq


In [2]:
stop_words = []
prefixes = []
postfixes = []
verb_roots = []
common_words = []
plural_singular = []

In [3]:
#######################################importing
def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def read_files():
    path = 'files/'

    #stop_words
    name = 'stop_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        stop_words.append(line)
    f.close()

    #prefixes
    name = 'prefixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        prefixes.append(line)
    f.close()

    #postfixes
    name = 'postfixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        postfixes.append(line)
    f.close()

    # verb_roots
    name = 'verb.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        verb_roots.append(line)
    f.close()

    #common_words
    name = 'common_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        common_words.append(line)
    f.close()

    #plural_singular
    name = 'plural_singular.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        plural_singular.append(line)
    f.close()

#######################################Normalization
def delete_punctuations(doc):
    punctuations = '؟،:؛!»«()[]*,{.}@!?،؛<>#$&!~"\|-_+'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    edited_doc = edited_doc.translate(str.maketrans('', '', string.punctuation))
    
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    for i in inverted_indexes:
        if len(i[1])/len(df) > 0.8:
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens): 
    for i in range(0, len(tokens)):
        # print(verb_roots)
        for root in verb_roots:
            if search(root[0], tokens[i]) or search(root[1], tokens[i]):
                common = False
                for c in common_words:
                    if c == tokens[i]:
                        common = True
                        # print("common")
                        break
                if common == False:
                    my_regex = r"\b^"+root[1]+r"\.*"
                    x = re.findall(my_regex, tokens[i])
 
                    if len(x) == 0:  
                        my_regex = r"\b...*"+root[1]+r"\b"
                        x1 = re.findall(my_regex, tokens[i])
                        if len(x1)==0:         
                            # print("root")
                            tokens[i] = tokens[i].replace(tokens[i], root[2])

    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def pluralToSingular(tokens):
    for i in range(0, len(tokens)):
        for ps in plural_singular:
            if search(ps[0], tokens[i]):
                tokens[i] = tokens[i].replace(tokens[i], ps[1])
    return tokens

#######################################tokenization
def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        doc = delete_stopWords(doc)

        doc = replaceArabicWords(doc)

        doc = delete_postfixes(doc)

        doc = delete_prefixes(doc)
        

        tokenized_doc = doc.split()
        tokenized_doc = replaceWithRoot(tokenized_doc)
        tokenized_doc = pluralToSingular(tokenized_doc)

        for token in tokenized_doc:
            if len(token) > 0:
                temp = []
                temp.append(token)
                temp.append(df.id[i])
                tokens.append(temp)
    tokens.sort()
    return tokens

#######################################create inverted index
def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            if len(token[0])>0:
                temp = []
                temp.append(token_temp)

                (unique, counts) = np.unique(doc_temp, return_counts=True)
                # frequencies = np.array((unique, counts), dtype='i4,f4').T.view(np.recarray)       
                frequencies = []      
                for i in range(len(unique)):
                    temp1 = []
                    temp1.append(unique[i])
                    temp1.append(counts[i])
                    frequencies.append(temp1)
                temp.append(frequencies)
                inverted_indexes.append(temp)
                doc_temp = []
                doc_temp.append(token[1])

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
    
    inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

#######################################calculate weights
def calculate_tfidf(df, inverted_indexes):
    doc_length = np.zeros(len(df)+1)
    len_df = len(df)
    for i in range(len(inverted_indexes)):
        if len(inverted_indexes[i][1]) > 0:
            # print(weighted_inverted_index[i])
            # print(len(inverted_indexes[i][1]))
            idf = math.log(len_df/len(inverted_indexes[i][1]), 10)
            inverted_indexes[i].append(idf)
            # print("idf = ", idf)
            for j in range(len(inverted_indexes[i][1])):
                tf = 1 + math.log(inverted_indexes[i][1][j][1], 10)
                # print("tf = ", tf)
                w = tf*idf
                # print("w : ",w)
                inverted_indexes[i][1][j][1] = w
                # if inverted_indexes[i][1][j][0] == 6545:
                #     print("6545: ", inverted_indexes[i][1][j])
                #     print("6545: ", inverted_indexes[i][0])
                # if inverted_indexes[i][1][j][0] == 6550:
                #     print("6550: ", inverted_indexes[i][1][j])
                #     print("6550: ", inverted_indexes[i][0])


                doc_length[inverted_indexes[i][1][j][0]] = doc_length[inverted_indexes[i][1][j][0]] + np.power(w, 2)
                
    doc_length =  np.sqrt(doc_length)
    return inverted_indexes, doc_length

#######################################create champion list
def create_championList(inverted_indexes, r):
    limit = r
    championList = []
    for i in range(len(inverted_indexes)):
        temp = []
        temp.append(inverted_indexes[i][0])
        temp_postingList = list(inverted_indexes[i][1])
        temp_postingList.sort(key=lambda x: x[1], reverse=True)

        temp1 = []
        r = limit
        if r > len(temp_postingList):
            r = len(temp_postingList)
        for j in range(r):
            # print(temp_postingList[j])
            temp1.append(temp_postingList[j])
        temp.append(temp1)
        if len(inverted_indexes[i]) == 3:
            temp.append(inverted_indexes[i][2])
        championList.append(temp)
    return championList

#######################################Searching
def search_token(inverted_indexes, token_name):
    # for token in inverted_indexes:
    #     if token[0] == token_name:
    #         # print("token*****: ", token[0])
    #         return token[1], token[2]
    if token_name in inverted_indexes:
        return inverted_indexes[token_name][0], inverted_indexes[token_name][1]
    print("this token not exist in our database")
    return -1, -1

def print_res(df, res):
    p_res = pd.DataFrame()

    p_res = pd.merge(df, res, on=['id'], how='inner')
    our_res = p_res[['id', 'rank', 'url']]
    our_res = our_res.sort_values(["rank"], ascending=False)

    print(our_res)
    

def binary_search(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return


    for i in range (0 ,len(tokenized_query)):
        docs, idf = search_token(inverted_indexes, tokenized_query[i])
        if docs != -1:
            temp_docs_id = pd.DataFrame(docs)
            docs_id = pd.DataFrame(docs_id.append(temp_docs_id))

    if len(docs_id) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1

    docs_id = docs_id[0].value_counts().reset_index()
    docs_id.columns = ['id', 'rank']
    print_res(df, docs_id)


def ranked_search(df, inverted_indexes, query, doc_length, k, using_heap):
    res = []
    docs_id = pd.DataFrame()
    
    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return -1

    (unique, counts) = np.unique(tokenized_query, return_counts=True)

    tokenized_query = np.asarray((unique, counts)).T
    # print(tokenized_query)

    doc_score_temp = []
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])
        # print(docs)


        if docs != -1:
            # print(docs)
            # print(idf)
            w_termInQuery = tf_query*idf
            # print("weight term in q: ", tf_query, idf, w_termInQuery)

            for i in range(len(docs)):
                temp = []
                temp.append(docs[i][0])
                # print("docs[i][1]: ",docs[i][1])
                temp.append(w_termInQuery*docs[i][1])
                doc_score_temp.append([docs[i][0], w_termInQuery*docs[i][1]])

    if len(doc_score_temp) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1
    doc_score_temp.sort()
    doc_score = []
    doc_score.append(doc_score_temp[0])
    for i in range(1, len(doc_score_temp)):
        if doc_score[len(doc_score)-1][0] == doc_score_temp[i][0]:
            doc_score[len(doc_score)-1][1] = doc_score[len(doc_score)-1][1] + doc_score_temp[i][1]
        else:
            doc_score.append(doc_score_temp[i])
    
    # for i in range(len(doc_score)):
    #     doc_score[i][1] = doc_score[i][1] / doc_length[doc_score[i][0]]

    res = []
    if using_heap == False:
        doc_score.sort(key=lambda x: x[1], reverse=True)  
        if len(doc_score) > k:
            for i in range (k):
                res.append(doc_score[i])
        else:
            res = doc_score
    if using_heap == True:
        hp = []
        for e in doc_score:
            hp.append((e[1], e[0]))

        nlargest = heapq.nlargest(k, hp)
        k_high_ranked = []
        for i in nlargest:
            k_high_ranked.append([i[1], i[0]])

        res = k_high_ranked

    docs_id = pd.DataFrame(res, columns=['id', 'rank'])
    print_res(df, docs_id)
    return doc_score

#######################################phase3 Clustering
def k_means(k, iteration):
    #select k centroids randomly
    centroids = []
    randoms = np.random.randint(1, len(inverted_index_perDoc) , size=(k))
    for i in range(len(randoms)):
        centroids.append(inverted_index_perDoc[randoms[i]-1][1])
    # print(centroids)

    max_index_row = 0
    cluster_docs = []       
    for it in range(iteration):
        print(it)
        doc_membership = np.zeros((len(df), k))
        for c in range(len(centroids)):
            print("******c= ",c)
            for term in centroids[c]:
            # for key,value in centroids[c].items():
                # print(dict_inverted_indexes)
                docs, idf = search_token(dict_inverted_indexes, term[0])
                if docs != -1:
                    for doc in docs:
                        doc_membership[doc[0]-1][c] = doc_membership[doc[0]-1][c] + term[1]*doc[1]
        #update centroids

        temp_index_max = max_index_row
        max_index_row = np.argmax(doc_membership, axis=1)
        if np.array_equal(temp_index_max, max_index_row):
            print("centroids not changed")
            cluster_name = np.argmax(doc_membership, axis=1)
            for c in range(len(centroids)):
                cluster_docs.append(np.where(cluster_name == c))

            return centroids, doc_membership, cluster_docs

        centroids = []
        for i in range(k):
            print("******k= ",i)
            doc_id_perCluster = np.where(max_index_row == i)
            # print(doc_id_perCluster)
            temp = []
            for doc_id in doc_id_perCluster[0]:
                # print(doc_id)
                # print("hey")
                for term in inverted_index_perDoc[doc_id][1]:
                    temp.append(term)
            centroids.append(temp)
        for i in range(k):
            # data_items = centroids[i].items()
            # data_list = list(data_items)
            df_centroid = pd.DataFrame(centroids[i], columns=['term','weight'])
            df_centroid = df_centroid.groupby('term', as_index=False)['weight'].mean()
            centroids[i] = df_centroid.to_numpy()
            # centroids[i] = dict(centroids[i]) 

        print(doc_membership)
        cluster_name = np.argmax(doc_membership, axis=1)
        for c in range(len(centroids)):
            cluster_docs.append(np.where(cluster_name == c))

    return centroids, doc_membership, cluster_docs

def ranked_search_withClustering(df, inverted_indexes, query, doc_length, k, using_heap, b2):
    res = []
    docs_id = pd.DataFrame()
    
    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return -1

    (unique, counts) = np.unique(tokenized_query, return_counts=True)

    tokenized_query = np.asarray((unique, counts)).T
    # print(tokenized_query)
    doc_score_temp = []
    docs_id = np.array
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])
        similarityToCentroids = np.zeros(len(centroids))
        w_termInQuery = tf_query*idf
        for c in range(len(centroids)):
            if tokenized_query[i][0] in centroids[c]:
                similarityToCentroids[c] = similarityToCentroids[c] + w_termInQuery*centroids[c][tokenized_query[i][0]]
    for b in range(b2):
        # print(similarityToCentroids)
        mostSimilar = np.argmax(similarityToCentroids)
        # print(mostSimilar)
        similarityToCentroids[mostSimilar] = 0

        if b == 0:
            docs_id = cluster_docs[mostSimilar][0]
        else:
            docs_id_temp = cluster_docs[mostSimilar][0]
            docs_id = np.append(docs_id, docs_id_temp)

    # print("*****")
    # print(docs_id)
    # print(len(docs_id))

    # print(similarityToCentroids)
    #search in docs
    doc_score_temp = []
    doc_score = []
    for i in range (0 ,len(tokenized_query)):
        # print(tokenized_query[i][0])
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])

        w_termInQuery = tf_query*idf
        # print(len(docs_id))
        for doc_id in docs_id:
            if tokenized_query[i][0] in inverted_index_perDoc[doc_id][1]:
                # print(doc_id)
                # print(w_termInQuery)
                # print(inverted_index_perDoc[doc_id][1][tokenized_query[i][0]])
                doc_score_temp.append([doc_id + 1, inverted_index_perDoc[doc_id][1][tokenized_query[i][0]] * w_termInQuery])
    if len(doc_score_temp) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1

    doc_score_temp.sort()
    # print(doc_score_temp)

    doc_score.append(doc_score_temp[0])
    # print(len(doc_score_temp))
    for i in range(1, len(doc_score_temp)):
        if doc_score[len(doc_score)-1][0] == doc_score_temp[i][0]:
            # print(doc_score_temp[i][0])
            # print(doc_score[len(doc_score)-1][1])
            doc_score[len(doc_score)-1][1] = doc_score[len(doc_score)-1][1] + doc_score_temp[i][1]
            # print(doc_score[len(doc_score)-1][1])
        else:
            doc_score.append(doc_score_temp[i])


    # for i in range(len(doc_score)):
    #     doc_score[i][1] = doc_score[i][1] / doc_length[doc_score[i][0]]

    res = []
    if using_heap == False:
        doc_score.sort(key=lambda x: x[1], reverse=True)  
        if len(doc_score) > k:
            for i in range (k):
                res.append(doc_score[i])
        else:
            res = doc_score
    if using_heap == True:
        hp = []
        for e in doc_score:
            hp.append((e[1], e[0]))

        nlargest = heapq.nlargest(k, hp)
        k_high_ranked = []
        for i in nlargest:
            k_high_ranked.append([i[1], i[0]])

        res = k_high_ranked

    docs_id = pd.DataFrame(res, columns=['id', 'rank'])
    print_res(df, docs_id)


    return doc_score



In [4]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")
read_files()

In [5]:
tokens = tokenize(df)

In [6]:
inverted_indexes = create_inverted_indexes(tokens)
inverted_indexes.pop(0)
inverted_indexes, doc_length = calculate_tfidf(df, inverted_indexes)
#normalize inverted_index
for i in range(len(inverted_indexes)):
    for j in range(len(inverted_indexes[i][1])):
        inverted_indexes[i][1][j][1] = inverted_indexes[i][1][j][1] / doc_length[inverted_indexes[i][1][j][0]]

championList = create_championList(inverted_indexes, 4)
# print(inverted_indexes[1])

##create dictionaries
dict_inverted_indexes = []
for i in range(len(inverted_indexes)):
    temp = []
    temp.append(inverted_indexes[i][0])
    temp.append([inverted_indexes[i][1], inverted_indexes[i][2]])
    dict_inverted_indexes.append(temp)
# inverted_indexes = dict(inverted_indexes)
# print(inverted_indexes['سلام'])
dict_championList = []
for i in range(len(championList)):
    temp = []
    temp.append(championList[i][0])
    temp.append([championList[i][1], championList[i][2]])
    dict_championList.append(temp)

dict_inverted_indexes = dict(dict_inverted_indexes)
dict_championList = dict(dict_championList)

#create inverted_index_perDoc
doc_term = []
for i in range(len(inverted_indexes)):
    for j in range(len(inverted_indexes[i][1])):
        doc_term.append([inverted_indexes[i][1][j][0], inverted_indexes[i][0], inverted_indexes[i][1][j][1]])

doc_term.sort()
# print(doc_term)
inverted_index_perDoc = []
temp = 0
temp = doc_term[0][0]
temp1 = []
for i in range(0, len(doc_term)):
    # print(inverted_index_perDoc[len(inverted_index_perDoc)][0])
    if temp == doc_term[i][0]:
        temp1.append([doc_term[i][1], doc_term[i][2]])
    else:
        inverted_index_perDoc.append([temp, temp1])
        temp = []
        temp1 = []

        temp = doc_term[i][0]
        temp1.append([doc_term[i][1], doc_term[i][2]])
inverted_index_perDoc.append([temp, temp1])

In [7]:
centroids, doc_membership, cluster_docs = k_means(10, 12)
for i in range(len(inverted_index_perDoc)):
    inverted_index_perDoc[i][1] = dict(inverted_index_perDoc[i][1])

for i in range(len(centroids)):
    centroids[i] = dict(centroids[i])

0
******c=  0
******c=  1
******c=  2
******c=  3
******c=  4
******c=  5
******c=  6
******c=  7
******c=  8
******c=  9
******k=  0
******k=  1
******k=  2
******k=  3
******k=  4
******k=  5
******k=  6
******k=  7
******k=  8
******k=  9
[[0.01908324 0.01294717 0.01774007 ... 0.01913441 0.01618659 0.02127122]
 [0.00279692 0.00736388 0.04367818 ... 0.0087927  0.0034343  0.02882471]
 [0.02610666 0.02146275 0.01702533 ... 0.02738324 0.03715067 0.02802724]
 ...
 [0.0370652  0.04358759 0.02321855 ... 0.02335309 0.02786364 0.0138877 ]
 [0.03523152 0.01799417 0.06205851 ... 0.03516612 0.00915432 0.01234305]
 [0.05541401 0.01910459 0.12102664 ... 0.01983228 0.00709569 0.01708847]]
1
******c=  0
******c=  1
******c=  2
******c=  3
******c=  4
******c=  5
******c=  6
******c=  7
******c=  8
******c=  9
******k=  0
******k=  1
******k=  2
******k=  3
******k=  4
******k=  5
******k=  6
******k=  7
******k=  8
******k=  9
[[0.55836209 0.62419231 0.62544982 ... 0.66750266 0.72626482 0.84454341]

In [8]:
query = "سلام"
# query_processing(df, inverted_indexes, query)

doc_score = ranked_search(df, dict_championList, query, doc_length, k = 10, using_heap = True)
# doc_score = ranked_search(df, championList, query, doc_length, k = 10, using_heap = False)
doc_score_temp = ranked_search_withClustering(df, dict_inverted_indexes, query, doc_length, k = 10, using_heap = True, b2 = 3)

doc_score = ranked_search(df, dict_inverted_indexes, query, doc_length, k = 10, using_heap = True)

binary_search(df, dict_inverted_indexes, query)

     id      rank                                                url
3  6480  0.339717  https://www.isna.ir/news/99121309809/پیامک-وزا...
1  6190  0.336205  https://www.isna.ir/news/99082416345/پیامک-وزی...
2  6248  0.307479  https://www.isna.ir/news/99091511010/درخواست-و...
0  1747  0.264576  https://www.isna.ir/news/99012011206/درخواست-م...
     id      rank                                                url
8  6480  0.339717  https://www.isna.ir/news/99121309809/پیامک-وزا...
7  6190  0.336205  https://www.isna.ir/news/99082416345/پیامک-وزی...
4  5437  0.252083  https://www.isna.ir/news/99010703106/بهبودی-حا...
3  5167  0.224681  https://www.isna.ir/news/98092317069/رییس-سازم...
0  1261  0.208939  https://www.isna.ir/news/98051206050/عذرخواهی-...
1  1744  0.184404  https://www.isna.ir/news/99011809763/ارائه-مشا...
2  1764  0.182862  https://www.isna.ir/news/99012615162/نسبت-به-ج...
5  5758  0.124448  https://www.isna.ir/news/99041914154/تنها-نیست...
6  5766  0.124448  https://www.isn

In [9]:
s = ['میرو']

print(replaceWithRoot(s))

['میرو']


In [10]:
# high_idf_tokens = []
# for i in range(len(inverted_indexes)):
#     if len(inverted_indexes[i]) == 3:
#         if inverted_indexes[i][2] > 3.84:
#             high_idf_tokens.append(inverted_indexes[i])
#             print(inverted_indexes[i])
# print(len(high_idf_tokens))

14]], 3.8450980400142565]
['\u200cریگان', [[84, 0.0843505564814353]], 3.8450980400142565]
['\u200cزابل', [[84, 0.0843505564814353]], 3.8450980400142565]
['\u200cزرقان', [[84, 0.0843505564814353]], 3.8450980400142565]
['\u200cزرین\u200cدشت', [[84, 0.0843505564814353]], 3.8450980400142565]
['\u200cزهک', [[84, 0.0843505564814353]], 3.8450980400142565]
['\u200cزیرکوه', [[84, 0.0843505564814353]], 3.8450980400142565]
['\u200cسادگی', [[3325, 0.10295239253486635]], 3.8450980400142565]
['\u200cساز', [[2416, 0.08730346912255484]], 3.8450980400142565]
['\u200cسال\u200c', [[3372, 0.1417291802499481]], 3.8450980400142565]
['\u200cسر', [[5614, 0.09900452156610326]], 3.8450980400142565]
['\u200cسرمربی', [[1449, 0.1926834044365903]], 3.8450980400142565]
['\u200cسلامتی', [[3188, 0.16900911601623267]], 3.8450980400142565]
['\u200cسمت', [[71, 0.13757678729488798]], 3.8450980400142565]
['\u200cسهمیه\u200cبندی', [[5102, 0.1073129185804554]], 3.8450980400142565]
['\u200cسوزی', [[3327, 0.2696987879329201]],

In [12]:
# df_invertedIndex = []
# for i in range(len(inverted_indexes)):
#     df_invertedIndex.append([inverted_indexes[i][0], pd.DataFrame(inverted_indexes[i][1], columns=['term','weight'])])

